In [1]:
import numpy as np
import pandas as pd

In [21]:
vendor_review_df=pd.read_json("../r2Data/yelp/yelp_academic_dataset_review.json",lines=True,nrows=200000)

In [22]:
vendor_review_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
199995,nRYJWQ6YVjcrZCEU_JWrIQ,uakTslg3Ay3C0816-5yNdQ,idf-eiurCrbsLRcH7c9zmw,5,0,0,0,Oakley's Bistro is a hidden gem for Indianapol...,2012-10-16 17:45:43
199996,4ylitTY7GmonnQDqhjB-eg,_fMw99otXOCu-Kzc-loMZA,v72HalBu5fQIFTD-oqhSgQ,5,0,0,0,"Let me just say, I'm glad my husband and I dec...",2016-08-18 00:23:31
199997,MnwNSr5h_t1uIlcj3aI_5w,ZfG1T_noQnbOdXnz4USs0Q,cvHY1RTAPSoH94ysP2_eyQ,5,1,1,1,I was coming back from the Farmers market toda...,2014-06-21 17:12:53
199998,3fp0uXGAjIqgza02_ObF5w,I0MMq4iYjDjERnSjp76HDw,cBp1fn2LXpSHe4VbTXLYEw,5,0,0,1,Absolutely perfect meal for me. The food is cl...,2017-09-06 17:54:23


In [18]:
vendor_review_df[vendor_review_df['business_id']=='vN6v8m4DO45Z4pp8yxxF_w']

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
498,WZIOWWb6xVwCA2-uJ47m8Q,XeS-0ONS5uoR_OfgZQebrA,vN6v8m4DO45Z4pp8yxxF_w,5,0,0,0,Best shrimp and grits in NOLA! Delicious fresh...,2017-01-02 14:24:15
852,nkjr-JtsZIeUvbX41_NNiw,I-wqWdMhZ-cgT4YPxY20gA,vN6v8m4DO45Z4pp8yxxF_w,5,0,0,0,Awesome! Loved this place. Great little breakf...,2018-06-26 17:21:36
1219,DvnGdnhLqG9DDAlAwCJ1fg,X8tAoBJPDqEhwwNuCQVa0w,vN6v8m4DO45Z4pp8yxxF_w,5,0,0,0,This place deserves a special mention.. it's n...,2018-01-09 21:25:17
1414,5xr-88dAcMOopVygu89O3A,d82e3N5c24CDA9Svi_t0fg,vN6v8m4DO45Z4pp8yxxF_w,4,0,0,0,If you beat the crowd you're in. Small but tha...,2011-09-25 21:27:10
1737,yYd9mySLhy2tgtl8CCXHhg,94ICh-yDZXOOGu-e33C6RA,vN6v8m4DO45Z4pp8yxxF_w,5,0,0,0,A short walk from my apartment I was told abou...,2017-11-06 16:37:45
...,...,...,...,...,...,...,...,...,...
48553,Pndld2ThF1IwENWIOt6pAQ,aLediJosDl1DlO50ZTNDmg,vN6v8m4DO45Z4pp8yxxF_w,4,0,0,0,The food was delicious! It was a little crampe...,2015-11-06 17:51:48
48607,yOO77knR7bG6W8p5-TCgow,JXv54EwhUoDVuYG5QgoMeA,vN6v8m4DO45Z4pp8yxxF_w,5,0,0,0,Banana Foster French Toast and Pulled Pork Ben...,2018-04-30 13:46:05
49221,OmIKGsQpoEgp0g7sCCTX6A,uJlAjOQlULnsTm52EBJUJA,vN6v8m4DO45Z4pp8yxxF_w,5,0,0,0,The food was absolutely amazing! The place wa...,2018-03-04 14:11:48
49556,bQoM_nqSqPpTCWa5mDVN9A,gpfpKWmQe0c33eXIvwid9w,vN6v8m4DO45Z4pp8yxxF_w,5,0,0,0,First I'll mention that this place is cash onl...,2011-01-10 19:14:46


In [23]:
vendor_business_info=pd.read_json("../r2Data/yelp/yelp_academic_dataset_business.json",lines=True)

In [24]:
vendor_business_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB
